## <p>
<font size='5' face='Georgia, Arial'>IIC2233 Apunte Programación Avanzada</font><br>
<font size='1'>Basado en: &copy; 2015 Karim Pichara - Christian Pieringer. Todos los derechos reservados. Editado por equipo docente IIC2233 2018-1 al 2023-2</font>
</p>

# Tabla de contenidos

1. [Encapsulamiento](#Encapsulamiento)
2. [_Properties_: `property`](#Properties:-property)
    1. [¿Para qué las `properties`?](#%C2%BFPara-qu%C3%A9-las-properties?)
    2. [Otras maneras de definir *properties*](#Otras-maneras-de-definir-properties)
    3. [Ejemplo: *caching* de páginas web](#Ejemplo:-caching-de-p%C3%A1ginas-web)
    4. [Ejemplo: definiendo una clase `Email`](#Ejemplo:-definiendo-una-clase-Email)
    5. [Ejemplo: definiendo figuras geométricas](#Ejemplo:-definiendo-figuras-geométricas)

# Encapsulamiento

En los lenguajes OOP, el encapsulamiento suele proveerse mediantes atributos _públicos_ o _privados_. En Python, **todos** los atributos y métodos de una clase son **públicos**, y el hecho de iniciar el nombre de un atributo o método con _underscore_ es una convención (y una buena práctica), pero no asegura un caracter privado de estos elementos. También hemos visto que existe una convención que permite _sugerir_ que un método o atributo es de uso únicamente interno. Esto se hace agregando un caracter _underscore_ (`_`) al inicio del atributo o método

Una consecuencia de tener atributos privados (o casi privados) es que si queremos modificarlos tenemos que, forzosamente, utilizar un método. En el paradigma OOP, se definen métodos específicos para **obtener el valor de un atributo (privado)**, y para **actualizar el valor de un atributo (privado)**. A estos métodos se llama respectivamente **getters** y **setters**.

Revisemos el ejemplo de la clase `Auto`.

In [2]:
class Auto:
    
    def __init__(self, marca, modelo, año, color, km):
        self.marca = marca
        self.modelo = modelo
        self.año = año
        self.color = color
        self._kilometraje = km
        self._ubicacion = (-33.45, -70.63)
        self.dueño = None

    def conducir(self, kms):
        self._kilometraje += kms
        self.__modificar_ubicacion()

    def vender(self, nuevo_dueño):
        self.dueño = nuevo_dueño

    def leer_odometro(self):
        return self._kilometraje

    def __modificar_ubicacion(self):
        print("Calcula nueva ubicación")
        self._ubicacion = (self._ubicacion[0] + 0.01, self._ubicacion[1] - 0.01)

Vemos que hay dos atributos "privados": `_kilometraje` y `_ubicacion`. Siguiendo la idea de OOP de utilizar _getters_ y *setters*, deberíamos definir dos métodos adicionales de la siguiente manera:

In [4]:
class Auto:
    
    def __init__(self, marca, modelo, año, color, km):
        self.marca = marca
        self.modelo = modelo
        self.año = año
        self.color = color
        self._kilometraje = km
        self._ubicacion = (-33.45, -70.63)
        self.dueño = None

    ## Método getter
    def get_kilometraje(self):
        return self._kilometraje
    
    ## Método setter
    def set_kilometraje(self, kms):
        if kms < 0:
            raise ValueError("No se puede establecer un kilometraje negativo")
        self._kilometraje = kms
        
    def conducir(self, kms):
        self._kilometraje += kms
        self._modificar_ubicacion()

    def vender(self, nuevo_dueño):
        self.dueño = nuevo_dueño

    def leer_odometro(self):
        return self._kilometraje

    def _modificar_ubicacion(self):
        print("Calcula nueva ubicación")
        self._ubicacion = (self._ubicacion[0] + 0.01, self._ubicacion[1] - 0.01)


Esto deberíamos repetirlo para cada atributo privado, de manera de cumplir con el principio de
encapsulamiento. Sin embargo, Python proveer un mecanismo para implementar encapsulamiento de
manera más sencillo. Éste es el mecanismo de **_properties_**.

En resumidas cuentas,  las **properties** realmente no cambian o agregan
nuevas cosas a la programación (que podrían hacerse con métodos más tradicionales), pero sí nos permiten escribir código más limpio y fácil de leer.

# *Properties*: `property`

En Python, una _property_ funciona como un atributo, pero sobre el cual podemos modificar su comportamiento cada vez que es leído (`get`), escrito (`set`), o eliminado (`del`). Al usar el mecanismo de _properties_ sobre un atributo, podemos ejecutar acciones de manera más limpia que invocando métodos explícitos para leer o modificar el valor de un objeto. Por supuesto debemos definir los métodos correspondientes *getter* y *setter*. Veremos que al utilizar estos métodos podemos agregar comportamiento adicional en cada caso.

## ¿Para qué las *properties*?

Tomemos un ejemplo sencillo en que deseamos definir una clase que modela un puente a través del cual puede pasar solo una cantidad máxima de personas por motivos de seguridad. Una manera inicial de implementarlo es mediante la siguiente clase.


In [5]:
class Puente:
    
    def __init__(self, maximo: int):
        self.maximo = maximo
        self.personas = 0
        
puente = Puente(maximo=10)
puente.personas += 7
print(f"Hay {puente.personas} personas en el puente.")
puente.personas += 5
print(f"Hay {puente.personas} personas en el puente.")
puente.personas -= 15
print(f"Hay {puente.personas} personas en el puente.")

Hay 7 personas en el puente.
Hay 12 personas en el puente.
Hay -3 personas en el puente.


En este ejemplo es fácil modificar la cantidad de personas en el puente de manera externa a él, simplemente accediendo al atributo (público) `personas`. Sin embargo, al permitir estas modificaciones, también permitimos que el puente quede con más personas de las permitidas, o bien con una cantidad negativa de personas. La manera obvia de corregir esto es agregando acciones al momento de modificar la cantidad de personas.

In [7]:
class Puente:
    
    def __init__(self, maximo):
        self.maximo = maximo
        self.personas = 0
        
        
puente = Puente(10)

if puente.personas + 7 > puente.maximo:
    puente.personas = puente.maximo
else:
    puente.personas += 7
print(f"Hay {puente.personas} personas en el puente.")

if puente.personas + 5 > puente.maximo:
    puente.personas = puente.maximo
else:
    puente.personas += 5
print(f"Hay {puente.personas} personas en el puente.")

if puente.personas - 15 < 0:
    puente.personas = 0
else:
    puente.personas -= 15
print(f"Hay {puente.personas} personas en el puente.")

# Esto es una solución, pero no es la mejor, ya que no es escalable, y no es reutilizable (depende
# exactamente de cierto funcionamiento interno); es por
# ello que surgen las properties al rescate.

Hay 7 personas en el puente.
Hay 10 personas en el puente.
Hay 0 personas en el puente.


Este código permite que el puente funcione de manera correcta. Sin embargo, el código ahora es **más complicado de leer y de mantener**. Cada vez que modificamos el valor del atributo `personas` _debemos recordar_ efectuar ciertas verificaciones que (1) entorpecen el flujo natural del programa y que (2) violan el principio de encapsulamiento ya que estas verificaciones deberían ser responsabilidad de la clase `Puente`, y no del código externo a `Puente`. La siguiente mejora agrega las verificaciones dentro de la clase, y **encapsula** el atributo `personas` dentro de métodos especialmente definidos para leerlo y modificarlo.

In [8]:
class Puente:
    
    def __init__(self, maximo):
        self.maximo = maximo
        self._personas = 0
        
    def contar(self):
        # Método getter, pero mal programado
        return self._personas
    
    def ingresar(self, p):
        # Método setter, pero mal programado
        if self._personas + p > self.maximo:
            self._personas = self.maximo
        elif self._personas + p < 0:
            self._personas = 0
        else:
            self._personas += p
            
            
puente = Puente(10)
puente.ingresar(7)
print(f"Hay {puente.contar()} personas en el puente.")
puente.ingresar(5)
print(f"Hay {puente.contar()} personas en el puente.")
puente.ingresar(-15)
print(f"Hay {puente.contar()} personas en el puente.")

Hay 7 personas en el puente.
Hay 10 personas en el puente.
Hay 0 personas en el puente.


Hemos movido el comportamiento de verificar las condiciones, hacia los métodos de la clase `Puente`. Hemos encapsulado el acceso y modificación del atributo `_personas`, que ahora es un atributo interno, dentro de los métodos `contar` e `ingresar` quienes tienen, respectivamente, la misión de leer u obtener (*getter*) y modificar o actualizar (*setter*) el valor del atributo interno `_personas`. Nuestro código es correcto y **más fácil de leer**. 

Algo que podemos lamentar es que, dentro de todo, la primera versión que leía y modificaba directamente el atributo `personas` tenía una sintaxis más sencilla y ahora, en cambio, debemos llamar métodos particulares `ingresar` y `contar`. Más aún, en el futuro, si decidimos cambiar los nombres de los métodos `ingresar` y `contar`, debemos buscar todas las veces en que los hemos usado fuera de la clase y modificarlos.

La mejor combinación de ambos mundos: encapsulamiento, y sintaxis más simple, la provee el mecanismo de **properties**. Para incorporarlo veamos el siguiente ejemplo:

In [31]:
class Puente:
    
    def __init__(self, maximo: int, personas: int = 0):
        self.maximo = maximo
        self._personas = personas  # Atributo "privado", por el uso de _
        
    @property
    def personas(self):
        # Este es el getter, pero bien programado. Nótese los decoradores (el @property)
        return self._personas

    @personas.setter
    def personas(self, p):
        # Este es el setter, pero bien programado. Nótese los decoradores (el @personas.setter)
        if p > self.maximo:
            self._personas = self.maximo
        elif p < 0:
            self._personas = 0
        else:
            self._personas = p
    
    @personas.deleter
    def personas(self):
        del self._personas

puente = Puente(maximo=10)
puente.personas = 7
print(f"Hay {puente.personas} personas en el puente.")
puente.personas += 5
print(f"Hay {puente.personas} personas en el puente.")
puente.personas -= 15
print(f"Hay {puente.personas} personas en el puente.")


Hay 7 personas en el puente.
Hay 10 personas en el puente.
Hay 0 personas en el puente.


El texto que empieza con `@` se conoce como **decorador**. Al escribir el decorador `@property` antes del método `personas`, estamos definiendo una _property_ de nombre `personas`. Esta _property_ se comporta como un atributo cuyo método _getter_ es precisamente el método `personas`. Adicionalmente podemos definir otro método como _setter_ (que nos permitirá modificar el valor de la *property*) y para eso le agregamos el decorador `@personas.setter`.

El código resultante que escribimos fuera de la clase es tan simple como en la primera versión, y además encapsula, dentro de los métodos _getter_ y _setter_ para `personas`, el comportamiento que verifica que las restricciones sobre el puente se cumplan.

## Otras maneras de definir _properties_

Si no parece tan clara la manera de definir _properties_ mediante decoradores, Python ofrece otra manera más explícita que provee el mismo comportamiento. Veamos la siguiente versión de la clase `Puente`:

In [7]:
class Puente:
    
    def __init__(self, maximo):
        self.maximo = maximo
        self._personas = 0
        
    def _get_personas(self):
        return self._personas

    def _set_personas(self, p):
        if p > self.maximo:
            self._personas = self.maximo
        elif p < 0:
            self._personas = 0
        else:
            self._personas = p
        
    personas = property(fget=_get_personas, fset=_set_personas)

    
puente = Puente(10)
puente.personas += 7
print(f"Hay {puente.personas} personas en el puente.")
puente.personas += 5
print(f"Hay {puente.personas} personas en el puente.")
puente.personas -= 15
print(f"Hay {puente.personas} personas en el puente.")

Hay 7 personas en el puente.
Hay 10 personas en el puente.
Hay 0 personas en el puente.


Esta vez hemos definido los métodos "pseudo-privados" `_get_personas` y `_set_personas`, y posteriormente hemos definido un atributo `personas` dentro de la clase, pero fuera de los otros métodos (*atributo de clase*). Este atributo se define como una [`property`](https://docs.python.org/3/library/functions.html#property) y se le indica que sus métodos _getter_ y _setter_ serán, respectivamente, `_get_personas` y `_set_personas`. 

El comportamiento es el mismo que en el caso anterior que usaba decoradores. Las verificaciones se encuentran encapsuladas en los métodos _getter_ y _setter_ asociados a la _property_ `personas`; la sintaxis es simple y no depende de los nombres internos de los métodos _getter_ y _setter_.

## Ejemplo: *caching* de páginas web

Supongamos que estamos implementando un navegador que hace consultas por páginas web. Cuando el navegador obtiene una página como resultado, guarda una copia de ella, incluyendo imágenes y otros elementos, en caso que el usuario quiera accederla de nuevo en el corto plazo, y de esta forma se evita tener que bajar todo el contenido otra vez. Este mecanismo se llama _caching_ y es un gran beneficio en el rendimiento de los navegadores web.

En el siguiente ejemplo, tenemos una clase `WebPage` con un atributo `__content` que corresponde al contenido de una página web. Si un usuario accede al contenido por primera vez, descargamos el contenido y lo guardamos. De esta forma, en los próximos accesos podemos retornar el contenido guardado sin la necesidad de bajarlo de nuevo.

In [26]:
from urllib.request import urlopen

class WebPage:

    def __init__(self, url: str):
        self.url = url
        self._content = None  # Recordar que en comparaciones, None es False, y cualquier otro valor es True (salvo 0 y False)
    
    @property
    def content(self):
        if not self._content:
            print("Obteniendo página web...")
            self._content = urlopen(self.url).read()
        return self._content

In [29]:
import time

page = WebPage("http://www.uc.cl")
now = time.time()                  # devuelve el tiempo en segundos
contenido_1 = page.content         # llama a la property 'content', que baja el contenido
print(f"Tiempo en obtener la página por primera vez: {time.time() - now} segundos.")

now = time.time()
contenido_2 = page.content         # llama a la property 'content', y esta vez no baja de nuevo el contenido
print(f"Tiempo en obtener la página por segunda vez: {time.time() - now} segundos.")

print(contenido_1 == contenido_2)  # True, ya que son el mismo objeto, y por ende, el mismo contenido

# Nótese que el tiempo de la primera llamada es variable, ya que depende de la velocidad de
# internet de cada uno, pero el tiempo de la segunda llamada es siempre 0, ya que no se baja el
# contenido de nuevo.

Obteniendo página web...
Tiempo en obtener la página por primera vez: 1.2124581336975098 segundos.
Tiempo en obtener la página por segunda vez: 0.0 segundos.
True


De esta manera hemos usado la *property* `content` para agregar un comportamiento al *getter*, de
manera que la segunda vez que ejecute no realice la consulta por segunda vez.

##### Toda property tiene un getter, pero no necesariamente un setter.

## Ejemplo: definiendo una clase `Email`

Este ejemplo utiliza la segunda forma de usar *properties*, que es definiendo los métodos y luego asignarlos a una variable usando `property`. Adicionalmente define un método *deleter* `_del_email` que se encarga de eliminar el atributo "privado" `self.__email`.

In [30]:
class Email:
    
    def __init__(self, address: str):
        self._email = address
        
    def _get_email(self):
        return self._email
        
    def _set_email(self, value):
        if '@' not in value:
            print("Esto no parece una dirección de correo.")
            # No se modifica self._email
        else:
            self._email = value
    
    def _del_email(self):
        print("¡Eliminaste el correo!")
        del self._email    

    email = property(_get_email, _set_email, _del_email, "Esta propiedad corresponde al correo.")
   

In [33]:
mail = Email("kp1@gmail.com")
print(mail.email)
mail.email = "kp2@gmail.com"
print(mail.email)
mail.email = "kp2.com"
print(mail.email)
del mail.email

kp1@gmail.com
kp2@gmail.com
Esto no parece una dirección de correo.
kp2@gmail.com
¡Eliminaste el correo!


La siguiente versión de la clase `Email` define la `property` utilizando la notación de decoradores. Es equivalente al ejemplo anterior.

In [1]:
class Email2:
    
    def __init__(self, address: str) -> None:
        self._email = address
    
    @property
    def email(self):
        return self._email
    
    @email.setter
    def email(self, value: str):
        if not isinstance(value, str):
            raise TypeError("El correo debe ser un string.")
        if '@' not in value:
            print("Esto no parece una dirección de correo. No se modifica el correo.")
        else:
            self._email = value
        

    @email.deleter
    def email(self):
        print("¡Eliminaste el correo!")
        del self._email

Se puede observar que el funcionamiento de `Email2` es equivalente al de `Email`:

In [50]:
mail = Email2("kp1@gmail.com")
print(mail.email)
mail.email = "kp2@gmail.com"
print(mail.email)
mail.email = "kp2.com"
print(mail.email)
mail.email = 123

# Ante el errror, luego todo lo que sigue no se ejecuta.
del mail.email



kp1@gmail.com
kp2@gmail.com
Esto no parece una dirección de correo. No se modifica el correo.
kp2@gmail.com


TypeError: El correo debe ser un string.

## Ejemplo: definiendo figuras geométricas

Este ejemplo utiliza la notación de decorador para construir *properties*. En este, usamos *properties* para calcular el área y perímetro de un `Cuadrado`, pero tambien usamos el `setter` para poder definir el área o perímetro de un `Cuadrado` y esto implica actualizar el atributo privado `lado`.

In [53]:
import math


class Cuadrado:
    def __init__(self, lado: float) -> None:
        self._lado = lado

    @property
    def area(self):
        return math.pow(self._lado, 2)

    @area.setter
    def area(self, nueva_area):
        self._lado = math.sqrt(nueva_area)

    @property
    def perimetro(self):
        return 4 * self._lado

    @perimetro.setter
    def perimetro(self, nuevo_perimetro):
        self._lado = nuevo_perimetro/4

In [55]:
ejemplo_1 = Cuadrado(5)
print(f"Lado: {ejemplo_1._lado}")
print(f"Area: {ejemplo_1.area}")
print(f"Perimetro: {ejemplo_1.perimetro}")

print("\nAjustando lado para que el área sea 144")
ejemplo_1.area = 144
print(f"Lado: {ejemplo_1._lado}")
print(f"Area: {ejemplo_1.area}")
print(f"Perimetro: {ejemplo_1.perimetro}")

print("\nAjustando lado para que el perimetro sea 51")
ejemplo_1.perimetro = 51
print(f"Lado: {ejemplo_1._lado}")
print(f"Area: {ejemplo_1.area}")
print(f"Perimetro: {ejemplo_1.perimetro}")

Lado: 5
Area: 25.0
Perimetro: 20

Ajustando lado para que el área sea 144
Lado: 12.0
Area: 144.0
Perimetro: 48.0

Ajustando lado para que el perimetro sea 51
Lado: 12.75
Area: 162.5625
Perimetro: 51.0


Ahora haremos el mismo proceso para modelar un `Círculo`. Nuevamente usamos *properties* para modelar el área y perímetro de un `Círculo` y con `setter` podemos definir el proceso de actualizar el `radio` en función del valor del área o perímetro deseado.

In [24]:
class Circulo:
    def __init__(self, radio: float) ->None:
        self._radio = radio

    @property
    def area(self):
        return math.pi * math.pow(self._radio, 2)

    @area.setter
    def area(self, nueva_area):
        self._radio = math.sqrt(nueva_area/math.pi)

    @property
    def perimetro(self):
        return 2 * math.pi * self._radio

    @perimetro.setter
    def perimetro(self, nuevo_perimetro):
        self._radio = nuevo_perimetro/(2*math.pi)


In [26]:
ejemplo_2 = Circulo(4)
print(f"Radio: {ejemplo_2._radio}")
print(f"Area: {ejemplo_2.area}")
print(f"Perimetro: {ejemplo_2.perimetro}")

print("\nAjustando radio para que el área sea 124")
ejemplo_2.area = 124
print(f"Radio: {ejemplo_2._radio}")
print(f"Area: {ejemplo_2.area}")
print(f"Perimetro: {ejemplo_2.perimetro}")

print("\nAjustando radio para que el perimetro sea 50")
ejemplo_2.perimetro = 50
print(f"Radio: {ejemplo_2._radio}")
print(f"Area: {ejemplo_2.area}")
print(f"Perimetro: {ejemplo_2.perimetro}")

Radio: 4
Area: 50.26548245743669
Perimetro: 25.132741228718345

Ajustando radio para que el área sea 124
Radio: 6.282549314314218
Area: 124.0
Perimetro: 39.47442154333028

Ajustando radio para que el perimetro sea 50
Radio: 7.957747154594767
Area: 198.94367886486916
Perimetro: 50.0
